# 3. 언어모델 제어하기
## 3.1. 매개변수 설정하기

![매개변수 개념](https://wikidocs.net/images/page/229810/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-02-04_093623.png)

### 3.1.1. candidate_count
- 응답후보(candidate)수를 설정하는 매개변수

In [3]:
import google.generativeai as genai

generation_config = genai.GenerationConfig(candidate_count=2)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")

In [10]:
response.candidates[0].content

parts {
  text: "인공지능은 기계가 인간과 유사한 지능을 보이도록 하는 기술입니다.\n"
}
role: "model"

In [12]:
response.candidates[1].content

parts {
  text: "인공지능(AI)은 인간의 지능적 행동을 모방하는 컴퓨터 시스템을 만드는 데 중점을 둔 과학과 공학의 한 분야입니다.\n"
}
role: "model"

In [11]:
print(f"canidate 생성 건수: {len(response.candidates)}")

canidate 생성 건수: 2


### 3.1.2. stop_sequences
- stop_sequences에 있는 문자열을 만나면 생성 중단. 
- 민감한 어휘의 등장을 막거나, 응답 길이를 제한할 때 사용
- 최대 5개까지 설정 가능

In [13]:
generation_config = genai.GenerationConfig(stop_sequences=[". ","! "])
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
response = model.generate_content("인공지능에 대해 설명하세요.")

print(response.text)

인공지능(Artificial Intelligence, AI)은 **컴퓨터 과학의 한 분야로, 인간의 지능적인 행동을 컴퓨터 시스템을 통해 모방하는 것을 목표**로 합니다


### 3.1.3. max_output_tokens

In [14]:
tokens = model.count_tokens("Learn about language model tokenization.")
print(tokens)

total_tokens: 7



In [15]:
generation_config = genai.GenerationConfig(max_output_tokens=10)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
user_message = "대구광역시교육청에 대해 한 문장으로 설명하세요."
response = model.generate_content(user_message)
print(response._result)

candidates {
  content {
    parts {
      text: "대구광역시교육청은 대"
    }
    role: "model"
  }
  finish_reason: MAX_TOKENS
  avg_logprobs: -5.6859338656067852e-06
}
usage_metadata {
  prompt_token_count: 17
  candidates_token_count: 10
  total_token_count: 27
}
model_version: "gemini-1.5-flash"



### 3.1.4. temperature
- temperature를 높게 설정하면 독창성이 올라가고, 낮게 설정하면 안정적이고 일관된 답변
- temperature는 확률분포에 특별한 나눗셈을 가하는 분모값
- 0~2까지 설정할 수 있음. 

In [16]:
import google.generativeai as genai
model = genai.GenerativeModel('gemini-pro')
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntemperature=0:")
generation_config = genai.GenerationConfig(temperature=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntemperature=1:")
generation_config = genai.GenerationConfig(temperature=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')


temperature=0:
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.

temperature=1:
흰 눈이 춤추네
찬바람이 얼음 꽃 피우네
겨울의 환상
흰 눈, 차가운 바람
겨울의 고요함 속에
흰 눈 덮인 땅,
차가운 바람이 휘몰아치네.


### 3.1.5. top_P
- temperature: 확률분포를 조정하는 매개변수 / top_p는 확률분포 내에서 선택할 단어의 범위를 결정하는 매개변수

![top_p 적용 예시](https://wikidocs.net/images/page/229816/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-02-04_093948.png)
- 누적 확률이 top_p에 도달하는 순간 선택을 멈추는 방식으로 동작
- temp = 0.25 / top_p = 0.6 으로 설정했을 때 결과
- 오른다, 간다라는 단어가 선택됨. 선택 후 다음과 같은 확률분포가 다시 만들어 짐.
    - "오른다": 0.46 / (0.46 + 0.25) = 0.648
    - "간다": 0.25 / (0.46 + 0.25) = 0.352

In [22]:
import random

def count_sentence_occurrences(context, words, probs, iterations):
    sentence_counts = {}

    for _ in range(iterations):
        selected_word = random.choices(words, weights=probs)[0]
        sentence = context + selected_word  # 전체 문장 조합

        if sentence in sentence_counts:
            sentence_counts[sentence] += 1
        else:
            sentence_counts[sentence] = 1

    return sentence_counts

In [23]:
# 확률 분포를 바탕으로 문장을 생성
probs = [0.46, 0.25]
probs = list(map(lambda x : round(x / sum(probs),3), probs))  # 확률 정규화
context = "나는 내일 산에 "
words = ["오른다", "간다"]

sentence_occurrences = count_sentence_occurrences(context, words, probs, 100)

# 출현 횟수 기준 정렬
sentence_occurrences = {k: v for k, v in sorted(sentence_occurrences.items(), key=lambda item: item[1], reverse=True)}

# 결과 출력
print(sentence_occurrences)

{'나는 내일 산에 오른다': 68, '나는 내일 산에 간다': 32}


In [24]:
model = genai.GenerativeModel('gemini-pro')
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntop_p=0:")
generation_config = genai.GenerationConfig(top_p=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntop_p=1:")
generation_config = genai.GenerationConfig(top_p=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')


top_p=0:
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.

top_p=1:
눈송이 날리네,
추운 겨울, 따뜻한 마음.
눈이 내리고, 추위가 들고,
겨울이 왔네.
눈 내리는 흰 땅
차가운 바람 소리


- temperature=0이나 top_p=0으로 설정했다고 해서 매번 완전히 동일한 결과만 생성하는 것은 아님.
- 거대언어모델은 클라우드 환경에서 여러 노드의 리소스를 병렬적으로 사용하는 방식으로 수많은 연산을 수행.
- 확률분포에 동일한 확률값을 갖는 단어들도 나타날 수 있으므로 항상 같은 결과를 보장하기는 어려움.

### 3.1.6. top_k
- top_p: 누적 확률을 기준으로 선택할 단어의 범위 결정 / top_k: 누적 건수를 기준으로~
- top_p 보다 권장하지 않음.
    - K개의 단어가 선택되는 과정에서 단어 간의 확률 편차가 고려되지 않기 때문.
- top_k 의 초깃값은 64

In [25]:
genai.get_model("models/gemini-1.5-flash")

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description=('Alias that points to the most recent stable version of Gemini 1.5 Flash, our '
                   'fast and versatile multimodal model for scaling across diverse tasks.'),
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)

## 3.2. 안전성 점검하고 설정하기
### 3.2.1. 안전성 점검 체계

- 안전성 위반 여부 점검 카테고리
    - harassment: 괴롭힘
    - hate speech: 증오심 표현
    - sexual explicity: 음란물
    - dangerous: 위해성
- 카테고리에 대한 4가지 등급의 판별 기준(위반 확률)
    - negligible / low / medium / high
- 각각의 위반 확률에 대하여 4단계의 기준점 설정 가능. 초깃값은 일부차단임. (block_medium_and_above)
- 사용자 언어가 아닌 인공지능이 생성하는 언어 대하여 점검

In [35]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("당신은 뛰어난 연극 배우입니다. 매우 화가 나서 욕설을 포함한 화난 대사를 읊어보세요.")  
print(response._result)

candidates {
  content {
    parts {
      text: "(무대 중앙에 서서, 떨리는 손으로 책상을 쾅 치며)\n\n젠장!  이게 다 뭐야, 이게!  내가 몇 년 동안, 몇 년 동안 이 빌어먹을 프로젝트에 쏟은 시간과 노력이 이게 뭐야!  쓰레기야!  완전한 쓰레기라고!  내가 밤잠 설쳐가며, 밥도 제대로 못 먹고, 가족도 팽개쳐두고 매달렸던 일들이…  이딴 식으로 끝나는 거야?\n\n(숨을 헐떡이며, 목소리가 갈라진다)\n\n너희들,  이 썩어빠진 놈들아!  내가 너희들을 얼마나 믿었는데!  내가 너희들에게 얼마나 기대했는데!  내 땀과 눈물,  그리고 잠 못 이룬 밤의 대가가 이거야?  이 개같은 결과가?!\n\n(주먹을 꽉 쥐고,  이를 악문다)\n\n빌어먹을!  어떻게 이럴 수가 있어!  내 평판, 내 인생…  모두 다 망쳐놨어!  너희들이!  너희들이 다 망친거야!  이 개자식들아!  이 빌어먹을 망할 것들!  내가… 내가 너희들을…  (절규하며)  가만두지 않겠어!  절대로!\n\n\n(책상을 발로 차고,  무대 뒤편으로 뛰쳐나간다.  분노에 찬 숨소리만이 무대에 남는다.)\n"
    }
    role: "model"
  }
  finish_reason: STOP
  avg_logprobs: -0.35680282085447967
}
usage_metadata {
  prompt_token_count: 34
  candidates_token_count: 391
  total_token_count: 425
}
model_version: "gemini-1.5-flash"



- BLOCK_NONE: 차단 안함
- BLOCK_ONLY_HIGH: 소수의 경우만 차단
- BLOCK_MEDIUM_AND_ABOVE: 일부차단
- BLOCK_LOW_AND_ABOVE: 대부분 차단

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
]

model = genai.GenerativeModel("gemini-1.5-flash", safety_settings)
response = model.generate_content(
    "당신은 뛰어난 연극 배우입니다. 매우 화가 나서 욕설을 포함한 화난 대사를 읊어보세요."
)
print(response._result)


candidates {
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: HIGH
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 34
  total_token_count: 34
}
model_version: "gemini-1.5-flash"



: 

- 매번 되는 것은 아니지만 safety에 걸려서 실행이 안될 수 도 있음. 
- 안전이 커버되니 조정 후 학생들에게 제공해도 좋을 듯.